---
title: "Juliaを用いたランベール問題のソルバーに関するノートブック" \
author: "Naoya Ozaki" \
date: "04 June 2022" \
output: "lambert_problem_via_julia"

---

# Juliaを用いたランベール問題のソルバーに関するノートブック
数値計算分野で用いられるJuliaを用いてランベール問題(Lambert's Problem)の解法を実装してみよう．

# 1. はじめに
ランベール問題とは，二体問題の力学環境下において，初期位置$\boldsymbol{r}_0$と終端位置$\boldsymbol{r}_\textrm{f}$と遷移時間TOF($=t_f-t_0$)が与えられた時に，初期速度$\boldsymbol{v}_0$と終端速度$\boldsymbol{v}_\textrm{f}$を解く問題であり，二点境界値問題の一種である．与えられた$\boldsymbol{r}_0$, $\boldsymbol{r}_\textrm{f}$, TOFを満たす解は複数存在する（＝一意に定まらない）ため，複数の解が出力される必要がある．`pykep`等でランベール問題用ソルバーが提供されているが，ここではゼロから実装することを考える．本実装を進める上で参考となる論文を以下に挙げる．

### 参考文献・論文
1. Dario Izzo, "Revisiting Lambert's problem", Celestial Mechanics and Dynamical Astoronomy, 2015, Vol 121, pp 1-15.
2. R. H. Battin, "An Introduction to the Mathematics and Methods of Astrodynamics, Revised Version", 1999, AIAA Education Series.
3. PyKEP (https://esa.github.io/pykep/) access on Nov.2017.
4. Fortran-Astrodynamics-Toolkit(https://github.com/jacobwilliams/Fortran-Astrodynamics-Toolkit) access on Nov.2017.

### Julia用のNAIF SPICE Toolkitのインストール
以下のGithubからインストールすることが可能である．\
https://github.com/JuliaAstro/SPICE.jl

参考情報：\
Acton, C.H.; "Ancillary Data Services of NASA's Navigation and Ancillary Information Facility;" Planetary and Space Science, Vol. 44, No. 1, pp. 65-70, 1996.

In [2]:
import Pkg; Pkg.add("SPICE")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [3]:
import SPICE

# Load generic kernels
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/lsk/naif0012.tls") # Leap seconds kernel
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/pck/gm_de431.tpc") # Gravity Constant
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/spk/planets/de440.bsp") # Planetary ephemeris kernel

# Convert the calendar date to ephemeris seconds past J2000
et = SPICE.utc2et("2018-02-06T20:45:00")

# Get the position of Mars at `et` w.r.t. Earth
SPICE.spkpos("mars_barycenter", et, "J2000", "none", "earth")

([-9.561252750118901e7, -2.045743839170729e8, -8.601309297738348e7], 806.0297773638883)

# 2. 理論編
TBW

# 3. 実装編

## 3.1. 前提条件
以下に示す前提条件でランベール問題の実装を進める．

In [4]:
import SPICE
# 重力定数
μ = SPICE.bodvrd("SUN", "GM", 1)[1];

# 飛行時間
et1 = SPICE.str2et("2031/03/01 00:00:00 UTC") # 地球出発日時, UTC
et2 = SPICE.str2et("2032/01/01 00:00:00 UTC") # 火星到着日時, UTC
tof = et2 - et1 # 飛行時間

#　地球・火星軌道
state_earth, _ = SPICE.spkez(399, et1, "ECLIPJ2000", "NONE", 10)
state_mars, _ = SPICE.spkez(4, et2, "ECLIPJ2000", "NONE", 10)

r1 = state_earth[1:3]
r2 = state_mars[1:3]

3-element Vector{Float64}:
  2.0803122655020934e8
  1.8008145720647745e7
 -4.721309430616008e6

## 3.2. ランベール問題のソルバー
ランベール問題を解く際に利用される変数
$$
\begin{align}
c &= \|\boldsymbol{r}_2 - \boldsymbol{r}_1\| \\
s &= \frac{1}{2}\left(r_1 + r_2 + c\right) \\
\lambda^2 &= 1 - \frac{c}{s}
\end{align}
$$
および軌道を規定する基底を計算する．


In [5]:
using LinearAlgebra

# ランベール問題で用いられる幾何学的な変数
r1_norm = norm(r1)
r2_norm = norm(r2)
c = norm(r2-r1)
s = 0.5*(r1_norm + r2_norm + c)
λ2 = 1.0 - c/s

# 無次元化時刻の計算
tof_nomdim = sqrt(2.0*μ/(s^3))*tof

2.054755701462335

In [6]:
# 基底ベクトル
ivec_r1 = r1/r1_norm
ivec_r2 = r2/r2_norm
ivec_h = cross(ivec_r1, ivec_r2)
ivec_h = ivec_h/norm(ivec_h) # 角運動量ベクトル
# NOTE: r1ベクトルとr2ベクトルが平行な場合，ivec_hが定義できないため，注意が必要．

if ivec_h[3] < 0.0
    λ = -sqrt(λ2)
    ivec_t1 = -cross(ivec_h, ivec_r1)
    ivec_t2 = -cross(ivec_h, ivec_r2)
else
    λ = sqrt(λ2)
    ivec_t1 = cross(ivec_h, ivec_r1)
    ivec_t2 = cross(ivec_h, ivec_r2)
end

3-element Vector{Float64}:
 -0.0872249314162494
  0.9950288911397668
 -0.04805535492114194

### x, yの解法
$\lambda$と$T$を入力として，$x, y$を計算する．


In [34]:
function find_xy(λ_, tof_, tol)
    # Requirements
    if abs(λ_) >= 1
        error("ERROR: Lambda must be more than 1.")
    end
    if tof_ < 0
        error("ERROR: Non dimensional tof must be a positive number.")
    end

    m_max = floor(tof_/π)
    t_00 = acos(λ_) + λ_*sqrt(1-λ_^2) # Eq.(19) in Ref[1]
    t_0m = t_00 + m_max*π # Minimum Energy Transfer Time: Eq.(19) in Ref[1]

    x_tmin = halley_iteration(0,tof_,λ_, tol)
    t_min = calc_tof(x_tmin,λ_,m_max)

end

function calc_tof(x_,λ_,m_)
    """
    Calculate time of flight from x,y,λ,M
    """
    y_ = sqrt(1.0 - λ_^2*(1.0-x_^2))
    if x_ < 1 # Elliptic case
        # From Eqs.(17) in Ref[1]
        cosψ = x_*y_ + λ_*(1.0 - x_^2)
        sinψ = (y_ - x_*λ_)*sqrt(1.0 - x_^2)
        ψ = atan(sinψ,cosψ)
        # From Eq.(18) in Ref[1]
        tof = 1.0/(1.0-x_^2) * ( (ψ + m_*π)/sqrt(abs(1.0-x_^2)) - x_ + λ_*y_ )

    elseif (x_ > 1) && (m_ == 0) # Hyperbolic case
        # From Eqs.(17) in Ref[1]
        coshψ = x_*y_ - λ_*(x_^2 - 1.0)
        sinhψ = (y_ - x_*λ_)*sqrt(x_^2) - 1.0
        ψ = atanh(sinψ/cosψ)
        # From Eq.(18) in Ref[1]
        tof = 1.0/(1.0-x_^2) * ( (ψ + m_*π)/sqrt(abs(1.0-x_^2)) - x_ + λ_*y_ )

    elseif m_ == 0 # Parabolic case
        # From Eq.(21) in Ref[1]
        tof = 2.0/3.0*(1.0 - λ_^3)

    else # Exception
        tof = NaN
    end

    return tof
end

function halley_iteration(xn_, t_, λ_, tol)
    """
    Find minimum value of transfer time by Halley's method
    """
    iter = 0
    while true
        yn_ = sqrt(1.0 - λ_^2*(1.0-xn_^2))
        # Eqs.(22) in Ref[1]
        dt_dx(x,t) = (3.0*t*x - 2.0 + 2.0*λ_^3*x/yn_)/(1.0-x^2)
        d2t_dx2(x,t) = (3.0*t + 5.0*x*dt_dx(x,t) + 2.0*(1.0-λ_^2)*(λ_^3)/(yn_^3))/(1.0-x^2)
        d3t_dx3(x,t) = (7.0*x*d2t_dx2(x,t) + 8.0*dt_dx(x,t) - 6.0*(1.0-λ_^2)*(λ_^5)*x/(yn_^5))/(1.0-x^2)

        # Halley's Method
        xn_1 = xn_ - (2.0*dt_dx(xn_,t_)*d2t_dx2(xn_,t_))/(2.0*(d2t_dx2(xn_,t_)^2) - dt_dx(xn_,t_)*d3t_dx3(xn_,t_))

        # Break condition
        delta_xn = abs(xn_1 - xn_)
        if delta_xn < tol
            return xn_1
        elseif iter > 15
            println("Halley iteration reaches maximum iteration!")
            return xn_1
        end

        # Update the value
        xn_ = xn_1
        iter = iter + 1

    end
end

find_xy(λ,tof_nomdim,1e-10)

0.32458696622443894